**No python code still!**

# Your First Database

## Intro

- Databases have advantage over flat files such as csv or excel files
- real-life entities become table
- redundancy reduced
- data integrity by *relations*
- use constraints, keys and referential integrity in order to assure data quality.

## Query information_schema with SELECT
information_schema is a meta-database that holds information about your current database. information_schema has multiple tables you can query with the known SELECT * FROM syntax:

tables: information about all tables in your current database
columns: information about all columns in all of the tables in your current database
...
In this exercise, you'll only need information from the 'public' schema, which is specified as the column table_schema of the tables and columns tables. The 'public' schema holds information about user-defined tables and databases. The other types of table_schema hold system information – for this course, you're only interested in user-defined stuff.

-- Get information on all table names in the current database, while limiting your query to the 'public' table_schema.
```
SELECT table_name 
FROM information_schema.tables
WHERE table_schema = 'public';
```
-- Now have a look at the columns in university_professors by selecting all entries in information_schema.columns that correspond to that table.
```
SELECT column_name, data_type 
FROM information_schema.columns 
WHERE table_name = 'university_professors' AND table_schema = 'public';
```

## Tables
- Multiple entity in a sigle table creates redundancy (professor, organization, university)
- We also create an affiliation table to connect professor and organization
- Entitiy relationship diagram

### CREATE your first few TABLEs
You'll now start implementing a better database model. For this, you'll create tables for the professors and universities entity types. The other tables will be created for you.

The syntax for creating simple tables is as follows:
```
CREATE TABLE table_name (
 column_a data_type,
 column_b data_type,
 column_c data_type
);
```
Attention: Table and columns names, as well as data types, don't need to be surrounded by quotation marks.

### Add COLUMN
To add columns you can use the following SQL query:
```
ALTER TABLE table_name
ADD COLUMN column_name data_type;
```

### INSERT
```
INSERT INTO organizations 
SELECT DISTINCT organization, 
    organization_sector
FROM university_professors;
```

```
INSERT INTO table_name (column_a, column_b)
VALUES ("value_a", "value_b");
```

### RENAME
```
ALTER TABLE table_name
RENAME COLUMN old_name TO new_name;
```

### DROP COLUMN
```
ALTER TABLE table_name
DROP COLUMN column_name;
```

### DROP TABLE
```
DROP TABLE table_name;
```

# Enforce data consistency with attribute constraints

### Integrity constraints
- Attribute constraints, e.g. data types on columns
- Key constraints, e.g. primary keys
- Referential integrity constraints, enforced through foreign keys

### Why constraints?
- Constraints give the data structure
- Constraints help with consistency, and thus data quality
- Data quality is a business advantage / data science prerequisite
- Enforcing is difficult, but PostgreSQL helps

### Casting
```
SELECT temperature * CAST(wind_speed AS integer) AS wind_chill
FROM weather;
```

### Working with data types
- Enforced on columns (i.e. attributes)
- Define the so-called "domain" of a column
- Define what operations are possible
- Enfore consistent storage of values

### e.g
```
CREATE TABLE students (
 ssn integer, 
 name varchar(64), 
 dob date, 
 average_grade numeric(3, 2), -- e.g. 5.54
 tuition_paid boolean 
);

ALTER TABLE students
ALTER COLUMN name 
TYPE varchar(128);

ALTER TABLE students
ALTER COLUMN average_grade
TYPE integer
-- Turns 5.54 into 6, not 5, before type conversion
USING ROUND(average_grade);
```

### Convert types USING a function
If you don't want to reserve too much space for a certain varchar column, you can truncate the values before converting its type.

For this, you can use the following syntax:
```
ALTER TABLE table_name
ALTER COLUMN column_name
TYPE varchar(x)
USING SUBSTRING(column_name FROM 1 FOR x)
```
You should read it like this: Because you want to reserve only x characters for column_name, you have to retain a SUBSTRING of every value, i.e. the first x characters of it, and throw away the rest. This way, the values will fit the varchar(x) requirement.

### The not-null constraint
- Disallow NULL values in a certain column
- Must hold true for the current state
- Must hold true for any future state

### e.g
```
CREATE TABLE students (
 ssn integer not null,
 lastname varchar(64) not null,
 home_phone integer,
 office_phone integer
);
```

- NULL!=NULL
- comparing NULL with NULL always results in a false value.

### add or remove NOT NULL
```
ALTER TABLE students 
ALTER COLUMN home_phone 
SET NOT NULL;

ALTER TABLE students 
ALTER COLUMN ssn 
DROP NOT NULL;
```

### The UNIQUE constraint
- Disallow duplicate values in a certain column
- Must hold true for the current state
- Must hold true for any future state

## e.g
```
CREATE TABLE table_name (
 column_name UNIQUE
);
 
ALTER TABLE table_name
ADD CONSTRAINT some_name UNIQUE(column_name);
--you have to give the constraint a name some_name.
```

# Uniquely identify records with key constraints


### What is a key?
- Attribute(s) that identify a record uniquely
- As long as attributes can be removed: superkey
- If no more attributes can be removed: minimal superkey or key

### e.g
K1 = {license_no}; K2 = {serial_no}; K3 = {model}; K4 = {make, year}
- K1 to 3 only consist of one attribute
- Removing either "make" or "year" from K4 would result in duplicates
- Only one candidate key can be the chosen key

### Get to know SELECT COUNT DISTINCT
Your database doesn't have any defined keys so far, and you don't know which columns or combinations of columns are suited as keys.

There's a simple way of finding out whether a certain column (or a combination) contains only unique values – and thus identifies the records in the table.

You already know the SELECT DISTINCT query from the first chapter. Now you just have to wrap everything within the COUNT() function and PostgreSQL will return the number of unique rows for the given columns:
```
SELECT COUNT(DISTINCT(column_a, column_b, ...))
FROM table;
```

### Identify keys with SELECT COUNT DISTINCT
There's a very basic way of finding out what qualifies for a key in an existing, populated table:

1. Count the distinct records for all possible combinations of columns. If the resulting number x equals the number of all rows in the table for a combination, you have discovered a superkey.

2. Then remove one column after another until you can no longer remove columns without seeing the number x decrease. If that is the case, you have discovered a (candidate) key.

The table professors has 551 rows. It has only one possible candidate key, which is a combination of two attributes. You might want to try different combinations using the "Run code" button. Once you have found the solution, you can submit your answer.

### Primary keys
- One primary key per database table, chosen from candidate keys
- Uniquely identifies records, e.g. for referencing in other tables
- Unique and not-null constraints both apply
- Primary keys are time-invariant: choose columns wisely!

### e.g
```
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);

OR

CREATE TABLE example (
    a integer,
    b integer,
    c integer,
    PRIMARY KEY (a, c)
);
```

### Add primary key
```
ALTER TABLE table_name
ADD CONSTRAINT some_name PRIMARY KEY (column_name)
```

### Surrogate key
- artificial primary keys
- primary keys on column that are there just for being a primary key
- e.g incremented numering as id in a table

### e.g making a surrogate key
```
ALTER TABLE cars
ADD COLUMN id serial PRIMARY KEY;
```
serial means increment

### with >1 columns
```
ALTER TABLE table_name
ADD COLUMN column_c varchar(256);

UPDATE table_name
SET column_c = CONCAT(column_a, column_b);
ALTER TABLE table_name
ADD CONSTRAINT pk PRIMARY KEY (column_c);
```

# Glue together tables with foreign keys


### Implementing relationships with foreign keys
- A foreign key (FK) points to the primary key (PK) of another table
- Domain of FK must be equal to domain of PK
- Each value of FK must exist in PK of the other table (FK constraint or "referential integrity")
- FKs are not actual keys, because duplicates and null values are allowed

### e.g
```
CREATE TABLE manufacturers (
 name varchar(255) PRIMARY KEY
);

INSERT INTO manufacturers 
VALUES ('Ford'), ('VW'), ('GM');
CREATE TABLE cars (
 model varchar(255) PRIMARY KEY,
 manufacturer_name integer REFERENCES manufacturers (name)
);

INSERT INTO cars 
VALUES ('Ranger', 'Ford'), ('Beetle', 'VW');
```

### Specifying foreign keys to existing tables
```
ALTER TABLE a 
ADD CONSTRAINT a_fkey FOREIGN KEY (b_id) REFERENCES b (id);
```

### JOIN tables linked by a foreign key
While foreign keys and primary keys are not strictly necessary for join queries, they greatly help by telling you what to expect. For instance, you can be sure that records referenced from table A will always be present in table B – so a join from table A will always find something in table B. If not, the foreign key constraint would be violated.

```
SELECT professors.lastname, universities.id, universities.university_city
FROM professors
JOIN universities
ON professors.university_id = universities.id
where universities.university_city = 'Zurich';
```

### n-m relations
#### How to implement N:M-relationships
- Create a table
- Add foreign keys for every connected table
- Add additional attributes
```
 CREATE TABLE affiliations (
 professor_id integer REFERENCES professors (id),
 organization_id varchar(256) REFERENCES organization (id),
 function varchar(256)
);
```
- No primary key!
- Possible PK = {professor_id, organization_id, function}
- But no need to create such a primary key, its a bit over the top.

### Populate the "professor_id" column
Here's a way to update columns of a table based on values in another table:
```
UPDATE table_a
SET column_to_update = table_b.column_to_update_from
FROM table_b
WHERE condition1 AND condition2 AND ...;
This query does the following:
```
For each row in table_a, find the corresponding row in table_b where condition1, condition2, etc., are met.
Set the value of column_to_update to the value of column_to_update_from (from that corresponding row).
The conditions usually compare other columns of both tables, e.g. table_a.some_column = table_b.some_column. Of course, this query only makes sense if there is only one matching row in table_b

### Referential integrity violations
Referential integrity from table A to table B is violated...

- if a record in table B that is referenced from a record in table A is deleted.
- if a record in table A referencing a non-existing record from table B is inserted.
- Foreign keys prevent violations!

### Dealing with violations
ON DELETE...(what to do if a referenced row is deleted?)

- NO ACTION: Throw an error (this is the default one)
- CASCADE: Delete all referencing records
- RESTRICT: Throw an error
- SET NULL: Set the referencing column to NULL
- SET DEFAULT: Set the referencing column to its default value

### Altering foreing key on delete actions

- Altering a key constraint doesn't work with ALTER COLUMN. Instead, you have to delete the key constraint and then add a new one with a different ON DELETE behavior.

- For deleting constraints, though, you need to know their name. This information is also stored in information_schema.

```
-- Identify the correct constraint name
SELECT constraint_name, table_name, constraint_type
FROM information_schema.table_constraints
WHERE constraint_type = 'FOREIGN KEY';

-- Drop the right foreign key constraint
ALTER TABLE affiliations
DROP CONSTRAINT affiliations_organization_id_fkey;

-- Add a new foreign key constraint from affiliations to organizations which cascades deletion
ALTER TABLE affiliations
ADD CONSTRAINT affiliations_organization_id_fkey FOREIGN KEY (organization_id) REFERENCES organizations (id) ON DELETE CASCADE;

-- Delete an organization 
DELETE FROM organizations 
WHERE id = 'CUREM';

-- Check that no more affiliations with this organization exist
SELECT * FROM organizations
WHERE id = 'CUREM';
```

### task:
- Count the number of total affiliations by university.
- Sort the result by that count, in descending order.

```
SELECT count(*), professors.university_id 
FROM affiliations
JOIN professors
ON affiliations.professor_id = professors.id
GROUP BY professors.university_id
order by count DESC;
```

### Join all the tables together
- find the university city of the professor with the most affiliations in the sector "Media & communication".

For this, you need to join all the tables, group by a column, and then use selection criteria to get only the rows in the correct sector.

```
-- Filter the table and sort it
SELECT COUNT(*), organizations.organization_sector, 
professors.id, universities.university_city
FROM affiliations
JOIN professors
ON affiliations.professor_id = professors.id
JOIN organizations
ON affiliations.organization_id = organizations.id
JOIN universities
ON professors.university_id = universities.id
where organizations.organization_sector = 'Media & communication'
GROUP BY organizations.organization_sector, 
professors.id, universities.university_city
order by count DESC;
```